In [1]:
import findspark
findspark.init('C:/spark-2.3.2-bin-hadoop2.7')

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, avg, col, concat, desc, explode, lit, min, max, split
from pyspark.sql.types import IntegerType

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import CountVectorizer, IDF, Normalizer, PCA, RegexTokenizer, StandardScaler, StopWordsRemover, VectorAssembler, StringIndexer, MinMaxScaler, PCA
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import re

In [3]:
data_dir = 'C:/Users/John/PycharmProjects/customer-attrition/data/'

In [4]:
spark = SparkSession.builder.appName("spark machine learning").getOrCreate()

In [5]:
df = spark.read.json(data_dir + 'Train_onetag_small.json')
df.persist()

DataFrame[Body: string, Id: bigint, Tags: string, Title: string, oneTag: string]

In [6]:
regexTokenizer = RegexTokenizer(inputCol='Body', outputCol='words', pattern="\\W")
df = regexTokenizer.transform(df)
df.head()

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'which

In [7]:
body_length = udf(lambda x: len(x), IntegerType())
df = df.withColumn('BodyLength', body_length('words'))

In [8]:
cv = CountVectorizer(inputCol='words', outputCol='TF', vocabSize = 1000)
cvmodel = cv.fit(df)
df = cvmodel.transform(df)
df.take(2)

[Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'whic

In [9]:
idf = IDF(inputCol = 'TF', outputCol = 'TFIDF')
idfmodel = idf.fit(df)
df = idfmodel.transform(df)
df.head()

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'which

In [10]:
indexer = StringIndexer(inputCol = 'oneTag', outputCol = 'label_tag')
df = indexer.fit(df).transform(df)

In [11]:
df.persist()

DataFrame[Body: string, Id: bigint, Tags: string, Title: string, oneTag: string, words: array<string>, BodyLength: int, TF: vector, TFIDF: vector, label_tag: double]

In [12]:
data = df.select(col('label_tag').alias('label'), col('TFIDF').alias('features'))
data.head()

Row(label=3.0, features=SparseVector(1000, {0: 0.0026, 1: 0.7515, 2: 0.1374, 3: 0.3184, 5: 0.3823, 8: 1.0754, 9: 0.3344, 15: 0.5899, 21: 1.8551, 28: 1.1263, 31: 1.1113, 35: 3.3134, 36: 1.2545, 43: 2.3741, 45: 2.3753, 48: 1.2254, 51: 1.1879, 57: 11.0264, 61: 2.8957, 71: 2.1945, 78: 1.6947, 84: 6.5898, 86: 1.6136, 94: 2.3569, 97: 1.8218, 99: 2.6292, 100: 1.9206, 115: 2.3592, 147: 5.4841, 152: 2.1116, 169: 2.6328, 241: 2.5745, 283: 3.2325, 306: 3.2668, 350: 6.2367, 490: 3.8893, 578: 3.6182, 759: 3.7771, 832: 8.8964}))

In [13]:
logreg = LogisticRegression(maxIter = 10, regParam = 0.0)

In [14]:
logmodel = logreg.fit(data)

In [15]:
logmodel.coefficientMatrix

DenseMatrix(301, 1000, [7.2356, 0.0372, 0.0333, 0.0894, -0.0442, 0.0287, 0.0018, 0.0007, ..., -0.0006, -0.0009, -0.0003, -0.0002, -0.0, -0.0003, -0.0015, -0.0005], 1)

In [16]:
logmodel.interceptVector

DenseVector([5.0624, 4.2809, 4.1836, 4.0456, 3.9815, 3.8424, 3.3918, 3.4562, 3.3316, 3.2418, 2.9428, 2.8218, 2.7839, 2.7625, 2.6392, 2.5983, 2.4539, 2.4447, 2.3916, 2.3566, 2.1003, 2.0631, 2.0567, 1.7878, 1.7815, 1.7789, 1.7183, 1.5344, 1.5141, 1.4106, 1.3633, 1.3618, 1.3407, 1.3321, 1.3387, 1.2438, 1.1902, 1.1985, 1.2037, 1.2022, 1.1798, 1.1327, 1.1006, 1.0406, 0.9521, 0.9417, 0.9192, 0.9164, 0.8901, 0.8584, 0.8452, 0.8359, 0.8296, 0.8064, 0.7944, 0.7899, 0.7819, 0.7776, 0.7598, 0.7628, 0.7327, 0.7291, 0.6964, 0.6557, 0.6597, 0.6572, 0.6451, 0.6439, 0.6062, 0.6087, 0.5191, 0.5071, 0.5063, 0.5012, 0.466, 0.4616, 0.4529, 0.4337, 0.4241, 0.4104, 0.406, 0.3852, 0.3536, 0.3461, 0.3453, 0.3236, 0.2877, 0.2839, 0.2742, 0.2597, 0.2447, 0.2194, 0.1946, 0.1855, 0.1849, 0.1718, 0.1684, 0.1625, 0.1454, 0.1231, 0.1156, 0.1063, 0.0996, 0.0669, 0.0713, 0.0213, 0.0143, 0.0013, 0.0025, -0.0089, -0.0199, -0.0291, -0.0283, -0.0409, -0.0398, -0.0501, -0.0721, -0.0735, -0.0711, -0.0729, -0.0839, -0.0852, 

In [17]:
logmodel.summary.accuracy

Py4JJavaError: An error occurred while calling o261.accuracy.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 39.0 failed 1 times, most recent failure: Lost task 2.0 in stage 39.0 (TID 194, localhost, executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.lang.reflect.Array.newInstance(Unknown Source)
	at java.io.ObjectInputStream.readArray(Unknown Source)
	at java.io.ObjectInputStream.readObject0(Unknown Source)
	at java.io.ObjectInputStream.readArray(Unknown Source)
	at java.io.ObjectInputStream.readObject0(Unknown Source)
	at java.io.ObjectInputStream.defaultReadFields(Unknown Source)
	at java.io.ObjectInputStream.readSerialData(Unknown Source)
	at java.io.ObjectInputStream.readOrdinaryObject(Unknown Source)
	at java.io.ObjectInputStream.readObject0(Unknown Source)
	at java.io.ObjectInputStream.readObject(Unknown Source)
	at org.apache.spark.serializer.JavaDeserializationStream.readObject(JavaSerializer.scala:75)
	at o

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 56042)
Traceback (most recent call last):
  File "C:\Anaconda_Udacity\Anaconda3\lib\socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "C:\Anaconda_Udacity\Anaconda3\lib\socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "C:\Anaconda_Udacity\Anaconda3\lib\socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "C:\Anaconda_Udacity\Anaconda3\lib\socketserver.py", line 696, in __init__
    self.handle()
  File "C:/spark-2.3.2-bin-hadoop2.7\python\pyspark\accumulators.py", line 265, in handle
    poll(accum_updates)
  File "C:/spark-2.3.2-bin-hadoop2.7\python\pyspark\accumulators.py", line 238, in poll
    if func():
  File "C:/spark-2.3.2-bin-hadoop2.7\python\pyspark\accumulators.py", line 242, in accum_updates
   